In [ ]:
import BPM
import BVP
from utils.errors import getErrors, printErrors, displayErrors
from AddedSignalProcess.signal_far_process_new import *

import scipy.signal._savitzky_golay
from AMTC import *
import torch
from AdaptiveNoise.adaptiveFilters import adap_filter_motion_class
from analysis.pipeline_ecgfitness_Motionclass import TestResult


In [ ]:
from scipy.interpolate import interp1d
def resample(ts, trace, resample_fs):
    # ts: time stamps
    # trace: signal
    # resample_fs: resample frequency
    f = interp1d(ts, trace, kind="linear")  # 定义插值函数
    duration = ts[-1] - ts[0]
    new_length = int(duration*resample_fs)+1
    ts_new = np.linspace(ts[0], ts[-1], new_length)
    trace_resample = f(ts_new)
    # return ts_new, trace_resample
    return trace_resample

In [ ]:
# 自适应动作分类选择滤波方法
from move_rPPG.SpectralSubtraction import spectrum_sub_motion_class
from AdaptiveNoise.adaptiveFilters import NLMS, AP
from MotionClassify.MotionClass import MotionCNN
from MotionClassify.MotionCNN import CNN

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
Motion_Class = torch.load('D:/motion_wu/MotionClassify/MotionClassTwoCNN.pth', map_location=device)
Motion_Class.eval()

# 数据库读取

In [ ]:
dataset_name = 'ecg_fitness'                              # 数据集名称
video_DIR = r"D:\ECG_FITNESS_RGB\ECG_FITNESS Signal_good_rppg"         # 视频数据目录的路径
BVP_DIR = r"D:\ECG_FITNESS_RGB\ECG_FITNESS Signal_good_rppg"

ecg_dataset = datasets.datasetFactory(dataset_name, videodataDIR=video_DIR, BVPdataDIR=BVP_DIR)  # 读取对应数据集的类方法
video_count = ecg_dataset.numVideos
video_Idx = [int(i) for i in range(video_count)]
print('All Videos: ', video_count)

# 分类降噪-HMM

In [ ]:
result_road = r"2025_分类降噪_0509_CHROM_origin.csv"
# result_road = r"D:\OneDrive\桌面\毕业论文\结果\20231229_分类降噪_HMM_方法测试.csv"
res = TestResult()
fps = 30
w_size = 20
stride = 1
for v in video_Idx:
    sigFileName = ecg_dataset.getSigFilename(v)
    print("\n## videoID: %d" % v)

    sigGT = ecg_dataset.readSigfile(sigFileName)            # 读取视频对应的BVP.gt数据
    bpmGT, timesGT = sigGT.getBPM(winsize=20)         # GT信息
    if v == 42:
        bpmGT = np.array(hampel_filter(bpmGT, 10, 2), dtype=np.float64)
    if v == 69:
        continue

    videoFileName = ecg_dataset.getVideoFilename(v)
    print(videoFileName)

    r_data = np.loadtxt(videoFileName[:-4] + "-1-R.txt")
    g_data = np.loadtxt(videoFileName[:-4] + "-1-G.txt")
    b_data = np.loadtxt(videoFileName[:-4] + "-1-B.txt")
    

    
    raw_rgb = np.array([[r_data, g_data, b_data]]).transpose((2, 0, 1))

    inx_data = np.loadtxt(videoFileName[:-4] + "-1-patch_index.txt")
    x_data = np.loadtxt(videoFileName[:-4] + "-1-X_Motion.txt")
    y_data = np.loadtxt(videoFileName[:-4] + "-1-Y_Motion.txt")
    z_data = np.loadtxt(videoFileName[:-4] + "-1-Z_Motion.txt")
    move_sig = np.array([inx_data, x_data, y_data], np.int32).transpose((1, 2, 0))

    data_name = videoFileName.split('\\')
    # if os.path.getsize("D:\\ECG_FITNESS_RGB\\Face_Loss\\" + data_name[-3] + data_name[-2] + ".txt"):
    #     face_lost = np.loadtxt("D:\\ECG_FITNESS_RGB\\Face_Loss\\" + data_name[-3] + data_name[-2] + ".txt", dtype=np.int16)
    # else:
    #     face_lost = None

    # 速度、加速度信号处理
    speed_x, speed_y, acc_x, acc_y = speed_acc_sig(move_sig, fps)  # 数组形式速度、加速度信号
    speed_x_sum, speed_y_sum, acc_x_sum, acc_y_sum, motion_sum = sum_speed_acc_sig(speed_x, speed_y, acc_x,
                                                                    acc_y, move_sig)  # patches速度加速度求和
    speed_x_sum = np.squeeze(speed_x_sum)
    speed_y_sum = np.squeeze(speed_y_sum)
    speed_z_sum = np.zeros_like(z_data)
    speed_z_sum[1:] = np.diff(z_data) * fps

    # if face_lost is not None:
    #     if face_lost.size == 1:
    #         face_lost = np.array([face_lost])
    #     speed_x_sum = lag_simulate(speed_x_sum, face_lost, 10)
    #     speed_y_sum = lag_simulate(speed_y_sum, face_lost, 10)
    #     speed_z_sum = lag_simulate(speed_z_sum, face_lost, 10)

    #     R_data = lag_simulate(r_data, face_lost, 10)
    #     G_data = lag_simulate(g_data, face_lost, 10)
    #     B_data = lag_simulate(b_data, face_lost, 10)
    #     raw_rgb = np.array([[R_data, G_data, B_data]]).transpose((2, 0, 1))

    windowed_rgb, _ = extraction.sig_windowing(raw_rgb, w_size, stride=1, fps=fps)     # 获取RGB原始窗信号

    # for m in ['CHROM', 'POS', 'LGI']:
    # for m in ['opt','GREEN', 'PBV', 'OMIT', 'PCA', 'ICA','CHROM','POS']:
    for m in ['opt','CHROM']:
        print("    Method:" + m)
        rppgs = None
        if m == 'opt':
            rppgs = BVP.RGB_sig_to_BVP(windowed_rgb, fps, device_type='cpu', method=find_optimal_signal)
        elif m == 'CHROM':
            rppgs = BVP.RGB_sig_to_BVP(windowed_rgb, fps, device_type='cpu', method=CHROME_DEHAAN)
        # elif m == 'POS':
        #     rppgs = BVP.RGB_sig_to_BVP(windowed_rgb, fps, device_type='cpu', method=cpu_POS, params={'fps': 'adaptive'})
        # elif m == 'LGI':
        #     rppgs = BVP.RGB_sig_to_BVP(windowed_rgb, fps, device_type='cpu', method=cpu_LGI)
        # elif m=='GREEN':
        #     rppgs = BVP.RGB_sig_to_BVP(windowed_rgb, fps, device_type='cpu', method=cpu_GREEN)
        # elif m == 'PBV':
        #     rppgs = BVP.RGB_sig_to_BVP(windowed_rgb, fps, device_type='cpu', method=cpu_PBV)
        # elif m == 'OMIT':
        #     rppgs = BVP.RGB_sig_to_BVP(windowed_rgb, fps, device_type='cpu', method=cpu_OMIT)
        # elif m == 'PCA':
        #     rppgs = BVP.RGB_sig_to_BVP(windowed_rgb, fps, device_type='cpu', method=cpu_PCA, params={'component':'second_comp'})
        # elif m == 'ICA':
        #     rppgs = BVP.RGB_sig_to_BVP(windowed_rgb, fps, device_type='cpu', method=cpu_ICA, params={'component':'second_comp'})
        


        bp_rppgs = apply_filter(rppgs, BPfilter, params={'oder': 6, 'minHz': 0.65, 'maxHz': 4.0, 'fps': fps})

        filter_rppgs = []

        # 自适应分类降噪滤波
        for fil in ["AP"]:

            params_xyz = {'miu':0.4, 'eps':0.001, 'weight':'random', 'time_history':5}
            ad_filter = None

            if fil == "NLMS":
                ad_filter = NLMS
            elif fil == "AP":
                params_xyz['miu'] = 0.2
                params_xyz['order'] = 8
                params_xyz['fc'] = 0.4
                ad_filter = AP

            win = 0
            fil_rppgs = []
            for win_rppgs in bp_rppgs:
                patch = 0
                patch_rppgs = []
                for patch_rppg in win_rppgs:
                    spx = speed_x_sum[int(win * stride * fps): int(win * stride * fps + w_size * fps)]
                    spy = speed_y_sum[int(win * stride * fps): int(win * stride * fps + w_size * fps)]
                    spz = speed_z_sum[int(win * stride * fps): int(win * stride * fps + w_size * fps)]
                    fx, tx, spec_x = scipy.signal.stft(spx, fs=fps, nperseg=200, noverlap=196, nfft=1024)
                    fy, ty, spec_y = scipy.signal.stft(spy, fs=fps, nperseg=200, noverlap=196, nfft=1024)
                    fz, tz, spec_z = scipy.signal.stft(spz, fs=fps, nperseg=200, noverlap=196, nfft=1024)
                    spec_x = np.abs(spec_x) / np.max(np.max(np.abs(spec_x)))
                    spec_y = np.abs(spec_y) / np.max(np.max(np.abs(spec_y)))
                    spec_z = np.abs(spec_z) / np.max(np.max(np.abs(spec_z)))
                    spec_xyz = spec_x + spec_y + spec_z

                    cla = MotionCNN(Motion_Class, spec_xyz, device=device)
                    if cla == 1:
                        if v in [74, 75]:
                            f_rppg = spectrum_sub_motion_class(patch_rppg, spec_xyz, fps, nperseg=200, overlap=196, nfft=1024, strong=30, road_list=[1, 0.5])
                        else:
                            f_rppg = spectrum_sub_motion_class(patch_rppg, spec_xyz, fps, nperseg=200, overlap=196, nfft=1024, strong=30, road_list=[1, 2])
                    else:
                        _, _, f_rppg = adap_filter_motion_class(patch_rppg, spx, spy, spz, fps, Method=ad_filter, params_x=params_xyz, params_y=params_xyz, params_z=params_xyz)

                    patch_rppgs.append(np.array(f_rppg))
                    patch += 1
                win += 1
                fil_rppgs.append(np.array(patch_rppgs))
            filter_rppgs.append({'rppg': fil_rppgs, 'filter': fil})    # 将每一种滤波方法

        for j in range(len(filter_rppgs)):
            filter_rppgs[j]['rppg'] = apply_filter(filter_rppgs[j]['rppg'], BPfilter, params={'oder': 6, 'minHz': 0.6, 'maxHz':4.0, 'fps': fps})

        # 心率提取
        for i in range(len(filter_rppgs)):
            filter_rppg = filter_rppgs[i]['rppg']
            rppg = complte_BVP_sig(filter_rppg, 1, fps).squeeze()    # rppg窗信号恢复成rPPG完整信号

            hmm_f, hmm_t, spectrum = signal.stft(rppg, fs=30, nperseg=300, noverlap=294, nfft=1024)    # stft时频分析
            spectrum = np.abs(spectrum) / np.max(np.max(np.abs(spectrum)))   # 时频图归一化

            # 调整gt心率与预测心率时刻长度一致
            hr_gt = bpmGT / 60
            TN = len(hmm_t)
            if len(hr_gt) < TN:
                hr_gt = scipy.ndimage.zoom(hr_gt, TN / len(hr_gt), order=3)
            if len(hr_gt) > TN:
                hr_gt = hr_gt[np.linspace(0, len(hr_gt) - 1, num=TN, dtype=np.int16)]

            # AMTC跟踪求心率
            tracker = AMTC()
            tracker.set_parameters(spectrum=spectrum, frequency=hmm_f, ti=hmm_t, num=1,
                                   threshold=0.05, ground_truth=hr_gt, scale=1)
            _, _, _ = tracker.run_AMTC(lam=5)
            roads = tracker.get_roads()
            road = roads[-1]
            road[0] = road[1]
            hr_hmm = np.array([hmm_f[i] * 60 for i in road])
            ti_index = [np.argmin(np.abs(ti - hmm_t)) for ti in timesGT]
            hr_hmm_es = hr_hmm[ti_index]
            ti_hmm_es = hmm_t[ti_index]
            RMSE, MAE, MAX, PCC, CCC, SNR = getErrors(filter_rppg, fps, hr_hmm_es, bpmGT, ti_hmm_es, timesGT)
            median_bpmES = hr_hmm_es
            mad_bpmES = np.median(median_bpmES)
            timesES = ti_hmm_es

            res.newDataSerie()
            res.addData('dataset', "ECG-fitness"); res.addData('method', m)
            res.addData('videoIdx', v);  res.addData('RMSE', RMSE)
            res.addData('MAE', MAE);  res.addData('MAX', MAX)
            res.addData('PCC', PCC);  res.addData('CCC', CCC)
            res.addData('SNR', SNR);  res.addData('bpmGT', bpmGT)
            res.addData('bpmES', median_bpmES);  res.addData('bpmES_mad', mad_bpmES)
            res.addData('timeGT', timesGT);  res.addData('timeES', timesES)
            res.addData('videoFilename', videoFileName);  res.addData('Move', videoFileName[-10])
            res.addData('Filter', filter_rppgs[i]['filter'])
            res.addDataSerie()
            print("        Filter:", filter_rppgs[i]['filter'])
            printErrors(RMSE, MAE, MAX, PCC, CCC, SNR)
res.saveResults(result_road)

# 分类降噪-FFT

In [ ]:
result_road = r"2025_分类降噪_FFT_0424.csv"
res = TestResult()
fps = 30
w_size = 20
stride = 1
for v in video_Idx:
    sigFileName = ecg_dataset.getSigFilename(v)
    print("\n## videoID: %d" % v)

    sigGT = ecg_dataset.readSigfile(sigFileName)            # 读取视频对应的BVP.gt数据
    bpmGT, timesGT = sigGT.getBPM(winsize=20)         # GT信息
    if v == 42:
        bpmGT = np.array(hampel_filter(bpmGT, 10, 2), dtype=np.float64)

    videoFileName = ecg_dataset.getVideoFilename(v)
    print(videoFileName)

    r_data = np.loadtxt(videoFileName[:-4] + "-1-R.txt")
    g_data = np.loadtxt(videoFileName[:-4] + "-1-G.txt")
    b_data = np.loadtxt(videoFileName[:-4] + "-1-B.txt")
    raw_rgb = np.array([[r_data, g_data, b_data]]).transpose((2, 0, 1))

    inx_data = np.loadtxt(videoFileName[:-4] + "-1-patch_index.txt")
    x_data = np.loadtxt(videoFileName[:-4] + "-1-X_Motion.txt")
    y_data = np.loadtxt(videoFileName[:-4] + "-1-Y_Motion.txt")
    z_data = np.loadtxt(videoFileName[:-4] + "-1-Z_Motion.txt")
    move_sig = np.array([inx_data, x_data, y_data], np.int32).transpose((1, 2, 0))

    data_name = videoFileName.split('\\')
    # if os.path.getsize("F:\\Data Set\\pyVHR\\ECG_FITNESS\\Face_Loss\\" + data_name[-3] + data_name[-2] + ".txt"):
    #     face_lost = np.loadtxt("F:\\Data Set\\pyVHR\\ECG_FITNESS\\Face_Loss\\" + data_name[-3] + data_name[-2] + ".txt", dtype=np.int16)
    # else:
    #     face_lost = None

    # 速度、加速度信号处理
    speed_x, speed_y, acc_x, acc_y = speed_acc_sig(move_sig, fps)  # 数组形式速度、加速度信号
    speed_x_sum, speed_y_sum, acc_x_sum, acc_y_sum, motion_sum = sum_speed_acc_sig(speed_x, speed_y, acc_x,
                                                                    acc_y, move_sig)  # patches速度加速度求和
    speed_x_sum = np.squeeze(speed_x_sum)
    speed_y_sum = np.squeeze(speed_y_sum)
    speed_z_sum = np.zeros_like(z_data)
    speed_z_sum[1:] = np.diff(z_data) * fps

    # if face_lost is not None:
    #     if face_lost.size == 1:
    #         face_lost = np.array([face_lost])
    #     speed_x_sum = lag_simulate(speed_x_sum, face_lost, 10)
    #     speed_y_sum = lag_simulate(speed_y_sum, face_lost, 10)
    #     speed_z_sum = lag_simulate(speed_z_sum, face_lost, 10)

    #     R_data = lag_simulate(r_data, face_lost, 10)
    #     G_data = lag_simulate(g_data, face_lost, 10)
    #     B_data = lag_simulate(b_data, face_lost, 10)
    #     raw_rgb = np.array([[R_data, G_data, B_data]]).transpose((2, 0, 1))

    windowed_rgb, _ = extraction.sig_windowing(raw_rgb, w_size, stride=1, fps=fps)     # 获取RGB原始窗信号

    for m in ['opt','GREEN', 'PBV', 'OMIT', 'PCA', 'ICA','CHROM','POS']:
        print("    Method:" + m)
        rppgs = None
        if m == 'opt':
            rppgs = BVP.RGB_sig_to_BVP(windowed_rgb, fps, device_type='cpu', method=find_optimal_signal)
        elif m == 'CHROM':
            rppgs = BVP.RGB_sig_to_BVP(windowed_rgb, fps, device_type='cpu', method=cpu_CHROM)
        elif m == 'POS':
            rppgs = BVP.RGB_sig_to_BVP(windowed_rgb, fps, device_type='cpu', method=cpu_POS, params={'fps': 'adaptive'})
        elif m == 'LGI':
            rppgs = BVP.RGB_sig_to_BVP(windowed_rgb, fps, device_type='cpu', method=cpu_LGI)
        elif m=='GREEN':
            rppgs = BVP.RGB_sig_to_BVP(windowed_rgb, fps, device_type='cpu', method=cpu_GREEN)
        elif m == 'PBV':
            rppgs = BVP.RGB_sig_to_BVP(windowed_rgb, fps, device_type='cpu', method=cpu_PBV)
        elif m == 'OMIT':
            rppgs = BVP.RGB_sig_to_BVP(windowed_rgb, fps, device_type='cpu', method=cpu_OMIT)
        elif m == 'PCA':
            rppgs = BVP.RGB_sig_to_BVP(windowed_rgb, fps, device_type='cpu', method=cpu_PCA, params={'component':'second_comp'})
        elif m == 'ICA':
            rppgs = BVP.RGB_sig_to_BVP(windowed_rgb, fps, device_type='cpu', method=cpu_ICA, params={'component':'second_comp'})
        
        bp_rppgs = apply_filter(rppgs, BPfilter, params={'oder': 6, 'minHz': 0.6, 'maxHz': 4.0, 'fps': fps})

        filter_rppgs = []

        # 自适应分类降噪滤波
        for fil in ["AP"]:

            params_xyz = {'miu':0.4, 'eps':0.001, 'weight':'random', 'time_history':5}
            ad_filter = None

            if fil == "NLMS":
                ad_filter = NLMS
            elif fil == "AP":
                params_xyz['miu'] = 0.2
                params_xyz['order'] = 8
                params_xyz['fc'] = 0.4
                ad_filter = AP

            win = 0
            fil_rppgs = []
            for win_rppgs in bp_rppgs:
                patch = 0
                patch_rppgs = []
                for patch_rppg in win_rppgs:
                    spx = speed_x_sum[int(win * stride * fps): int(win * stride * fps + w_size * fps)]
                    spy = speed_y_sum[int(win * stride * fps): int(win * stride * fps + w_size * fps)]
                    spz = speed_z_sum[int(win * stride * fps): int(win * stride * fps + w_size * fps)]
                    fx, tx, spec_x = scipy.signal.stft(spx, fs=fps, nperseg=200, noverlap=196, nfft=1024)
                    fy, ty, spec_y = scipy.signal.stft(spy, fs=fps, nperseg=200, noverlap=196, nfft=1024)
                    fz, tz, spec_z = scipy.signal.stft(spz, fs=fps, nperseg=200, noverlap=196, nfft=1024)
                    spec_x = np.abs(spec_x) / np.max(np.max(np.abs(spec_x)))
                    spec_y = np.abs(spec_y) / np.max(np.max(np.abs(spec_y)))
                    spec_z = np.abs(spec_z) / np.max(np.max(np.abs(spec_z)))
                    spec_xyz = spec_x + spec_y + spec_z

                    cla = MotionCNN(Motion_Class, spec_xyz, device=device)
                    if cla == 1:
                        if v in [74, 75]:
                            f_rppg = spectrum_sub_motion_class(patch_rppg, spec_xyz, fps, nperseg=200, overlap=196, nfft=1024, strong=30, road_list=[1, 0.5])
                        else:
                            f_rppg = spectrum_sub_motion_class(patch_rppg, spec_xyz, fps, nperseg=200, overlap=196, nfft=1024, strong=30, road_list=[1, 2])
                    else:
                        _, _, f_rppg = adap_filter_motion_class(patch_rppg, spx, spy, spz, fps, Method=ad_filter, params_x=params_xyz, params_y=params_xyz, params_z=params_xyz)

                    patch_rppgs.append(np.array(f_rppg))
                    patch += 1
                win += 1
                fil_rppgs.append(np.array(patch_rppgs))
            filter_rppgs.append({'rppg': fil_rppgs, 'filter': fil})    # 将每一种滤波方法

        for j in range(len(filter_rppgs)):
            filter_rppgs[j]['rppg'] = apply_filter(filter_rppgs[j]['rppg'], BPfilter, params={'oder': 6, 'minHz': 0.6, 'maxHz':4.0, 'fps': fps})

        # 心率提取
        for i in range(len(filter_rppgs)):
            filter_rppg = filter_rppgs[i]['rppg']
            rppg = complte_BVP_sig(filter_rppg, 1, fps).squeeze()    # rppg窗信号恢复成rPPG完整信号
            rppg = np.array([[rppg]]).transpose((2,1,0))

            window_rppg, timesES = extraction.sig_windowing(rppg, 20, stride=1, fps=fps)     # 获取RGB原始窗信号
            win_rppg = [w_rppg[0] for w_rppg in window_rppg]

            bpmES = BPM.BVP_to_BPM(win_rppg, fps, minHz=0.6, maxHz=4.0)  # 根据BVP信号获取每一秒的BPM信号（心跳次数）
            median_bpmES, mad_bpmES = BPM.multi_est_BPM_median(bpmES)
            RMSE, MAE, MAX, PCC, CCC, SNR = getErrors(win_rppg, fps, median_bpmES, bpmGT, timesES, timesGT)


            res.newDataSerie()
            res.addData('dataset', "ECG-fitness"); res.addData('method', m)
            res.addData('videoIdx', v);  res.addData('RMSE', RMSE)
            res.addData('MAE', MAE);  res.addData('MAX', MAX)
            res.addData('PCC', PCC);  res.addData('CCC', CCC)
            res.addData('SNR', SNR);  res.addData('bpmGT', bpmGT)
            res.addData('bpmES', median_bpmES);  res.addData('bpmES_mad', mad_bpmES)
            res.addData('timeGT', timesGT);  res.addData('timeES', timesES)
            res.addData('videoFilename', videoFileName);  res.addData('Move', videoFileName[-10])
            res.addData('Filter', filter_rppgs[i]['filter'])
            res.addDataSerie()
            print("        Filter:", filter_rppgs[i]['filter'])
            printErrors(RMSE, MAE, MAX, PCC, CCC, SNR)
res.saveResults(result_road)

# 原始BP-HMM

In [ ]:
result_road = r"2025_原始BP_HMM.csv"
res = TestResult()
fps = 30
w_size = 20
stride = 1
for v in video_Idx:
    sigFileName = ecg_dataset.getSigFilename(v)
    print("\n## videoID: %d" % v)

    sigGT = ecg_dataset.readSigfile(sigFileName)            # 读取视频对应的BVP.gt数据
    bpmGT, timesGT = sigGT.getBPM()         # GT信息
    if v == 42:
        bpmGT = np.array(hampel_filter(bpmGT, 10, 2), dtype=np.float64)

    videoFileName = ecg_dataset.getVideoFilename(v)
    print(videoFileName)

    r_data = np.loadtxt(videoFileName[:-4] + "-1-R.txt")
    g_data = np.loadtxt(videoFileName[:-4] + "-1-G.txt")
    b_data = np.loadtxt(videoFileName[:-4] + "-1-B.txt")
    raw_rgb = np.array([[r_data, g_data, b_data]]).transpose((2, 0, 1))

    inx_data = np.loadtxt(videoFileName[:-4] + "-1-patch_index.txt")
    x_data = np.loadtxt(videoFileName[:-4] + "-1-X_Motion.txt")
    y_data = np.loadtxt(videoFileName[:-4] + "-1-Y_Motion.txt")
    z_data = np.loadtxt(videoFileName[:-4] + "-1-Z_Motion.txt")
    move_sig = np.array([inx_data, x_data, y_data], np.int32).transpose((1, 2, 0))

    data_name = videoFileName.split('\\')
    # if os.path.getsize("F:\\Data Set\\pyVHR\\ECG_FITNESS\\Face_Loss\\" + data_name[-3] + data_name[-2] + ".txt"):
    #     face_lost = np.loadtxt("F:\\Data Set\\pyVHR\\ECG_FITNESS\\Face_Loss\\" + data_name[-3] + data_name[-2] + ".txt", dtype=np.int16)
    # else:
    #     face_lost = None

    # 速度、加速度信号处理
    speed_x, speed_y, acc_x, acc_y = speed_acc_sig(move_sig, fps)  # 数组形式速度、加速度信号
    speed_x_sum, speed_y_sum, acc_x_sum, acc_y_sum, motion_sum = sum_speed_acc_sig(speed_x, speed_y, acc_x,
                                                                    acc_y, move_sig)  # patches速度加速度求和
    speed_x_sum = np.squeeze(speed_x_sum)
    speed_y_sum = np.squeeze(speed_y_sum)
    speed_z_sum = np.zeros_like(z_data)
    speed_z_sum[1:] = np.diff(z_data) * fps

    # if face_lost is not None:
    #     if face_lost.size == 1:
    #         face_lost = np.array([face_lost])
    #     speed_x_sum = lag_simulate(speed_x_sum, face_lost, 10)
    #     speed_y_sum = lag_simulate(speed_y_sum, face_lost, 10)
    #     speed_z_sum = lag_simulate(speed_z_sum, face_lost, 10)

    #     R_data = lag_simulate(r_data, face_lost, 10)
    #     G_data = lag_simulate(g_data, face_lost, 10)
    #     B_data = lag_simulate(b_data, face_lost, 10)
    #     raw_rgb = np.array([[R_data, G_data, B_data]]).transpose((2, 0, 1))

    windowed_rgb, _ = extraction.sig_windowing(raw_rgb, w_size, stride=1, fps=fps)     # 获取RGB原始窗信号

    for m in ['opt','GREEN', 'PBV', 'OMIT', 'PCA', 'ICA','CHROM','POS']:
        print("    Method:" + m)
        rppgs = None
        if m == 'opt':
            rppgs = BVP.RGB_sig_to_BVP(windowed_rgb, fps, device_type='cpu', method=find_optimal_signal)
        elif m == 'CHROM':
            rppgs = BVP.RGB_sig_to_BVP(windowed_rgb, fps, device_type='cpu', method=cpu_CHROM)
        elif m == 'POS':
            rppgs = BVP.RGB_sig_to_BVP(windowed_rgb, fps, device_type='cpu', method=cpu_POS, params={'fps': 'adaptive'})
        elif m == 'LGI':
            rppgs = BVP.RGB_sig_to_BVP(windowed_rgb, fps, device_type='cpu', method=cpu_LGI)
        elif m=='GREEN':
            rppgs = BVP.RGB_sig_to_BVP(windowed_rgb, fps, device_type='cpu', method=cpu_GREEN)
        elif m == 'PBV':
            rppgs = BVP.RGB_sig_to_BVP(windowed_rgb, fps, device_type='cpu', method=cpu_PBV)
        elif m == 'OMIT':
            rppgs = BVP.RGB_sig_to_BVP(windowed_rgb, fps, device_type='cpu', method=cpu_OMIT)
        elif m == 'PCA':
            rppgs = BVP.RGB_sig_to_BVP(windowed_rgb, fps, device_type='cpu', method=cpu_PCA, params={'component':'second_comp'})
        elif m == 'ICA':
            rppgs = BVP.RGB_sig_to_BVP(windowed_rgb, fps, device_type='cpu', method=cpu_ICA, params={'component':'second_comp'})

        filter_rppgs = []

        # 带通滤波
        for fil in ["AP"]:

            fil_rppgs = apply_filter(rppgs, BPfilter, params={'oder': 6, 'minHz': 0.65, 'maxHz': 4.0, 'fps': fps})
            filter_rppgs.append({'rppg': fil_rppgs, 'filter': fil})    # 将每一种滤波方法

        # 心率提取
        for i in range(len(filter_rppgs)):
            filter_rppg = filter_rppgs[i]['rppg']
            rppg = complte_BVP_sig(filter_rppg, 1, fps).squeeze()    # rppg窗信号恢复成rPPG完整信号

            hmm_f, hmm_t, spectrum = signal.stft(rppg, fs=30, nperseg=300, noverlap=294, nfft=1024)    # stft时频分析
            spectrum = np.abs(spectrum) / np.max(np.max(np.abs(spectrum)))   # 时频图归一化

            # 调整gt心率与预测心率时刻长度一致
            hr_gt = bpmGT / 60
            TN = len(hmm_t)
            if len(hr_gt) < TN:
                hr_gt = scipy.ndimage.zoom(hr_gt, TN / len(hr_gt), order=3)
            if len(hr_gt) > TN:
                hr_gt = hr_gt[np.linspace(0, len(hr_gt) - 1, num=TN, dtype=np.int16)]

            # AMTC跟踪求心率
            tracker = AMTC()
            tracker.set_parameters(spectrum=spectrum, frequency=hmm_f, ti=hmm_t, num=1,
                                   threshold=0.05, ground_truth=hr_gt, scale=1)
            _, _, _ = tracker.run_AMTC(lam=5)
            roads = tracker.get_roads()
            road = roads[-1]
            road[0] = road[1]
            hr_hmm = np.array([hmm_f[i] * 60 for i in road])
            ti_index = [np.argmin(np.abs(ti - hmm_t)) for ti in timesGT]
            hr_hmm_es = hr_hmm[ti_index]
            ti_hmm_es = hmm_t[ti_index]
            RMSE, MAE, MAX, PCC, CCC, SNR = getErrors(filter_rppg, fps, hr_hmm_es, bpmGT, ti_hmm_es, timesGT)
            median_bpmES = hr_hmm_es
            mad_bpmES = np.median(median_bpmES)
            timesES = ti_hmm_es


            res.newDataSerie()
            res.addData('dataset', "ECG-fitness"); res.addData('method', str(m))
            res.addData('videoIdx', v);  res.addData('RMSE', RMSE)
            res.addData('MAE', MAE);  res.addData('MAX', MAX)
            res.addData('PCC', PCC);  res.addData('CCC', CCC)
            res.addData('SNR', SNR);  res.addData('bpmGT', bpmGT)
            res.addData('bpmES', median_bpmES);  res.addData('bpmES_mad', mad_bpmES)
            res.addData('timeGT', timesGT);  res.addData('timeES', timesES)
            res.addData('videoFilename', videoFileName);  res.addData('Move', videoFileName[-10])
            res.addData('Filter', filter_rppgs[i]['filter'])
            res.addDataSerie()
            print("        Filter:", filter_rppgs[i]['filter'])
            printErrors(RMSE, MAE, MAX, PCC, CCC, SNR)
res.saveResults(result_road)

# 原始BP-FFT

In [ ]:
from datetime import datetime
result_road = f"{datetime.now():%Y%m%d}_原始BP_FFT_resample.csv"
res = TestResult()
fps = 30
w_size = 20
stride = 1
for v in video_Idx:
    sigFileName = ecg_dataset.getSigFilename(v)
    print("\n## videoID: %d" % v)

    sigGT = ecg_dataset.readSigfile(sigFileName)            # 读取视频对应的BVP.gt数据
    bpmGT, timesGT = sigGT.getBPM()         # GT信息
    if v == 42:
        bpmGT = np.array(hampel_filter(bpmGT, 10, 2), dtype=np.float64)

    videoFileName = ecg_dataset.getVideoFilename(v)
    print(videoFileName)

    r_data = np.loadtxt(videoFileName[:-4] + "-1-R.txt")
    g_data = np.loadtxt(videoFileName[:-4] + "-1-G.txt")
    b_data = np.loadtxt(videoFileName[:-4] + "-1-B.txt")
    
    ts = np.loadtxt(videoFileName[:-4] + ".csv",delimiter=',',usecols=0)
    ts = ts/1000
    
    r_data = resample(ts, r_data, fps)
    g_data = resample(ts, g_data, fps)
    b_data = resample(ts, b_data, fps)
    
    raw_rgb = np.array([[r_data, g_data, b_data]]).transpose((2, 0, 1))

    inx_data = np.loadtxt(videoFileName[:-4] + "-1-patch_index.txt")
    x_data = np.loadtxt(videoFileName[:-4] + "-1-X_Motion.txt")
    y_data = np.loadtxt(videoFileName[:-4] + "-1-Y_Motion.txt")
    z_data = np.loadtxt(videoFileName[:-4] + "-1-Z_Motion.txt")
    move_sig = np.array([inx_data, x_data, y_data], np.int32).transpose((1, 2, 0))

    data_name = videoFileName.split('\\')
    # if os.path.getsize("F:\\Data Set\\pyVHR\\ECG_FITNESS\\Face_Loss\\" + data_name[-3] + data_name[-2] + ".txt"):
    #     face_lost = np.loadtxt("F:\\Data Set\\pyVHR\\ECG_FITNESS\\Face_Loss\\" + data_name[-3] + data_name[-2] + ".txt", dtype=np.int16)
    # else:
    #     face_lost = None

    # 速度、加速度信号处理
    speed_x, speed_y, acc_x, acc_y = speed_acc_sig(move_sig, fps)  # 数组形式速度、加速度信号
    speed_x_sum, speed_y_sum, acc_x_sum, acc_y_sum, motion_sum = sum_speed_acc_sig(speed_x, speed_y, acc_x,
                                                                    acc_y, move_sig)  # patches速度加速度求和
    speed_x_sum = np.squeeze(speed_x_sum)
    speed_y_sum = np.squeeze(speed_y_sum)
    speed_z_sum = np.zeros_like(z_data)
    speed_z_sum[1:] = np.diff(z_data) * fps

    # if face_lost is not None:
    #     if face_lost.size == 1:
    #         face_lost = np.array([face_lost])
    #     speed_x_sum = lag_simulate(speed_x_sum, face_lost, 10)
    #     speed_y_sum = lag_simulate(speed_y_sum, face_lost, 10)
    #     speed_z_sum = lag_simulate(speed_z_sum, face_lost, 10)

    #     R_data = lag_simulate(r_data, face_lost, 10)
    #     G_data = lag_simulate(g_data, face_lost, 10)
    #     B_data = lag_simulate(b_data, face_lost, 10)
    #     raw_rgb = np.array([[R_data, G_data, B_data]]).transpose((2, 0, 1))

    windowed_rgb, _ = extraction.sig_windowing(raw_rgb, w_size, stride=1, fps=fps)     # 获取RGB原始窗信号

    for m in ['opt','GREEN', 'PBV', 'OMIT', 'PCA', 'ICA','CHROM','POS']:
        print("    Method:" + m)
        rppgs = None
        if m == 'opt':
            rppgs = BVP.RGB_sig_to_BVP(windowed_rgb, fps, device_type='cpu', method=find_optimal_signal)
        elif m == 'CHROM':
            rppgs = BVP.RGB_sig_to_BVP(windowed_rgb, fps, device_type='cpu', method=cpu_CHROM)
        elif m == 'POS':
            rppgs = BVP.RGB_sig_to_BVP(windowed_rgb, fps, device_type='cpu', method=cpu_POS, params={'fps': 'adaptive'})
        elif m == 'LGI':
            rppgs = BVP.RGB_sig_to_BVP(windowed_rgb, fps, device_type='cpu', method=cpu_LGI)
        elif m=='GREEN':
            rppgs = BVP.RGB_sig_to_BVP(windowed_rgb, fps, device_type='cpu', method=cpu_GREEN)
        elif m == 'PBV':
            rppgs = BVP.RGB_sig_to_BVP(windowed_rgb, fps, device_type='cpu', method=cpu_PBV)
        elif m == 'OMIT':
            rppgs = BVP.RGB_sig_to_BVP(windowed_rgb, fps, device_type='cpu', method=cpu_OMIT)
        elif m == 'PCA':
            rppgs = BVP.RGB_sig_to_BVP(windowed_rgb, fps, device_type='cpu', method=cpu_PCA, params={'component':'second_comp'})
        elif m == 'ICA':
            rppgs = BVP.RGB_sig_to_BVP(windowed_rgb, fps, device_type='cpu', method=cpu_ICA, params={'component':'second_comp'})

        filter_rppgs = []

        # 带通滤波
        for fil in ["AP"]:

            fil_rppgs = apply_filter(rppgs, BPfilter, params={'oder': 6, 'minHz': 0.65, 'maxHz': 4.0, 'fps': fps})
            filter_rppgs.append({'rppg': fil_rppgs, 'filter': fil})    # 将每一种滤波方法

        # 心率提取
        for i in range(len(filter_rppgs)):
            filter_rppg = filter_rppgs[i]['rppg']
            rppg = complte_BVP_sig(filter_rppg, 1, fps).squeeze()    # rppg窗信号恢复成rPPG完整信号
            rppg = np.array([[rppg]]).transpose((2,1,0))

            window_rppg, timesES = extraction.sig_windowing(rppg, 20, stride=1, fps=fps)     # 获取RGB原始窗信号
            win_rppg = [w_rppg[0] for w_rppg in window_rppg]

            bpmES = BPM.BVP_to_BPM(win_rppg, fps, minHz=0.6, maxHz=4.0)  # 根据BVP信号获取每一秒的BPM信号（心跳次数）
            median_bpmES, mad_bpmES = BPM.multi_est_BPM_median(bpmES)
            RMSE, MAE, MAX, PCC, CCC, SNR = getErrors(win_rppg, fps, median_bpmES, bpmGT, timesES, timesGT)


            res.newDataSerie()
            res.addData('dataset', "ECG-fitness"); res.addData('method', str(m))
            res.addData('videoIdx', v);  res.addData('RMSE', RMSE)
            res.addData('MAE', MAE);  res.addData('MAX', MAX)
            res.addData('PCC', PCC);  res.addData('CCC', CCC)
            res.addData('SNR', SNR);  res.addData('bpmGT', bpmGT)
            res.addData('bpmES', median_bpmES);  res.addData('bpmES_mad', mad_bpmES)
            res.addData('timeGT', timesGT);  res.addData('timeES', timesES)
            res.addData('videoFilename', videoFileName);  res.addData('Move', videoFileName[-10])
            res.addData('Filter', filter_rppgs[i]['filter'])
            res.addDataSerie()
            print("        Filter:", filter_rppgs[i]['filter'])
            printErrors(RMSE, MAE, MAX, PCC, CCC, SNR)
res.saveResults(result_road)